<a href="https://colab.research.google.com/github/legacyai/legacyai_notebooks/blob/master/sentence2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone -b tft_typing https://github.com/legacyai/tf-transformers.git

!pip install sentencepiece

!pip install tensorflow-text

!pip install transformers

!pip install wandb

!pip install datasets

fatal: destination path 'tf-transformers' already exists and is not an empty directory.


In [2]:
import sys
sys.path.append("/content/tf-transformers/src/")

import tensorflow as tf
import tensorflow_text as tf_text

In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
from tf_transformers.core import Trainer
trainer = Trainer(distribution_strategy='tpu', tpu_address='colab')

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.36.245.242:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.36.245.242:8470


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


1. Create **Tokenizer**

In [5]:
from tf_transformers.models import AlbertTokenizerTFText

# tokenizer_layer = AlbertTokenizerTFText.from_pretrained("albert-base-v2", add_special_tokens=False, pack_model_inputs=True, max_length=256)
tokenizer_layer = AlbertTokenizerTFText.from_pretrained("albert-base-v2", add_special_tokens=False)

cls_token_id = tokenizer_layer.cls_token_id
sep_token_id = tokenizer_layer.sep_token_id

INFO:absl:Loading albert-base-v2 tokenizer to /tmp/tftransformers_tokenizer_cache/albert-base-v2/spiece.model


In [6]:
tokenizer_layer({'text': ['This is a sentence', 'this is']})

<tf.RaggedTensor [[48, 25, 21, 5123], [48, 25]]>

In [7]:
tokenizer_layer.pack_model_inputs

False

2. Create **Dataset**

In [90]:
from random import shuffle
from tf_transformers.data import TFReader
from tf_transformers.utils import tf_utils

#delim_regex_pattern = r'\.\s?\n?'
delim_regex_pattern = '\. '

window_length = 10
minimum_sentences = 4
batch_size = 512
max_seq_length = 256

def split_fn(item):
  text_splitted = tf_text.regex_split(
      item['text'][0], delim_regex_pattern, keep_delim_regex_pattern=' ', name='split'
  )
  return text_splitted

def filter_sentence_by_count(item):
  n_sentences = tf.cast(item.row_splits[1], tf.int32)
  if tf.less(n_sentences, minimum_sentences):
    return tf.constant(False)
  return tf.constant(True)

def _large_compatible_negative(tensor_type):
    """Large negative number as Tensor.
    This function is necessary because the standard value for epsilon
    in this module (-1e9) cannot be represented using `tf.float16`.
    Args:
      tensor_type: a dtype to determine the type.
    Returns:
      a large negative number.
    """
    if tensor_type == tf.float16:
        return tf.float16.min
    return -1e9


def filter_empty_string(sentences):
        """This will ensure, if any of the sentence in list of sentences is '' or' ', empty string,
        that will be filtered out"""
        sentences = sentences[0]
        valid_string_indexes = tf.squeeze(tf.where(tf.not_equal(tf.strings.length(sentences), 0)), axis=1)
        sentences = tf.gather(sentences, valid_string_indexes)

        # We need RaggedTensor
        sentences = tf.RaggedTensor.from_tensor(tf.expand_dims(sentences, axis=0))
        return sentences

def attention_mask_square(nd):
    """1's in the lower triangle, counting from the lower right corner.

    Same as tf.matrix_band_part(tf.ones([nd, ns]), -1, ns-nd), but doesn't produce garbage on TPUs.
    """
    dtype = tf_utils.get_dtype()
    ns = nd
    i = tf.range(nd)[:, None]
    j = tf.range(ns)
    m = i >= j - ns + nd
    return tf.cast(m, dtype)

def prepare_centre_neighbour_sentences(item):

  n_sentences = tf.cast(item.row_splits[1], tf.int32)
  # 1 - n_sentences-1
  centre_index = tf.random.uniform(minval=1, maxval = n_sentences-1, shape=(), dtype=tf.int32)

  min_window_boundary = tf.maximum(0, centre_index-window_length)
  max_window_boundary = tf.minimum(centre_index+window_length, n_sentences)

  neighbour_indexes = tf.squeeze(tf.where(tf.not_equal(tf.range(min_window_boundary, max_window_boundary), centre_index)), axis=1)

  # Randomly sample it for each example with different number of neighbours
  # n_neighbours = tf.random.uniform(minval=1, maxval = tf.shape(neighbour_indexes)[0], shape=(), dtype=tf.int32)
  # neighbour_index_ids = tf.random.uniform(minval=0, maxval = tf.shape(neighbour_indexes)[0], shape=(n_neighbours,), dtype=tf.int32)
  # neighbour_indexes = tf.gather(neighbour_indexes, neighbour_index_ids)


  centre_sentence = tf.gather(item[0], centre_index)
  neighbour_sentence = tf.gather(item[0], neighbour_indexes)
  # Repeat n-times
  n = tf.shape(neighbour_sentence)[0]
  centre_sentence = tf.repeat(centre_sentence, [n])


  # vec = tf.expand_dims(tf.range(batch_size), 0)

  # We use this same vector for all similiar sentences
  # Then we calculate scores (M x M transpose)
  # use this scores to generate mask for softmax
  # vec = tf.stop_gradient(tf.random.uniform(shape=(1, 32))) # keep a unique random
  # multiply = tf.stack([n, 1], axis=0)
  # tensor = tf.cast(tf.tile(vec, multiply), tf.float32)

  return {'centre_sentence': centre_sentence, 'neighbour_sentence': neighbour_sentence}

def shuffle_locally(item):
  indices = tf.range(start=0, limit=tf.shape(item['centre_sentence'])[0], dtype=tf.int32)
  shuffled_indices = tf.random.shuffle(indices)

  centre_sentence = tf.gather(item['centre_sentence'] , shuffled_indices)
  neighbour_sentence = tf.gather(item['neighbour_sentence'] , shuffled_indices)
  # mask = tf.gather(item['mask'], shuffled_indices)

  result = {}
  result['centre_sentence'] = centre_sentence
  result['neighbour_sentence'] = neighbour_sentence
  # result['mask'] = mask

  return result

def create_softmax_mask(item):
    scores = tf.matmul(item['mask'], item['mask'], transpose_b=True)
    # Replace similar scores place with -1e-9
    scores_mask = tf.where(tf.equal(scores, tf.linalg.diag_part(scores)), _large_compatible_negative(scores.dtype), tf.cast(0.0, scores.dtype))
    # Reset only diagonal entries back to 1.0
    scores_mask = tf.linalg.set_diag(scores_mask, tf.zeros(shape=(batch_size)), name='make_diagonal_one')
    item['mask'] = scores_mask
    return item

def text_to_features(item):
    """Convert item a tuple (src, target) into features"""
    result = {}
    input_ids = tokenizer_layer({'text': item['centre_sentence']})[:max_seq_length-2] # 2 for CLS and SEP
    # Add CLS and SEP
    input_ids = tf_text.combine_segments([input_ids], start_of_sequence_id=cls_token_id, end_of_segment_id=sep_token_id)[0]
    input_ids, input_mask = tf_text.pad_model_inputs(input_ids, max_seq_length=max_seq_length)
    input_type_ids = tf.zeros_like(input_mask)

    result['centre_input_ids'] = input_ids
    result['centre_input_mask'] = input_mask
    result['centre_input_type_ids'] = input_type_ids

    input_ids = tokenizer_layer({'text': item['neighbour_sentence']})[:max_seq_length-2] # 2 for CLS and SEP
    # Add CLS and SEP
    input_ids = tf_text.combine_segments([input_ids], start_of_sequence_id=cls_token_id, end_of_segment_id=sep_token_id)[0]
    input_ids, input_mask = tf_text.pad_model_inputs(input_ids, max_seq_length=max_seq_length)
    input_type_ids = tf.zeros_like(input_mask)

    result['neighbour_input_ids'] = input_ids
    result['neighbour_input_mask'] = input_mask
    result['neighbour_input_type_ids'] = input_type_ids

    # labels = {'mask': item['mask']}
    labels = {}

    return result, labels


# def text_to_features(item):
#     """Convert item a tuple (src, target) into features"""
#     result = {}
#     centre = tokenizer_layer({'text': item['centre_sentence']})
#     neighbour = tokenizer_layer({'text': item['neighbour_sentence']})
  
#     result = {}

#     result['centre_input_ids'] = centre['input_ids']
#     result['centre_input_mask'] = centre['input_mask']
#     result['centre_input_type_ids'] = centre['input_type_ids']

#     result['neighbour_input_ids'] = neighbour['input_ids']
#     result['neighbour_input_mask'] = neighbour['input_mask']
#     result['neighbour_input_type_ids'] = neighbour['input_type_ids']

#     labels = {}

#     return result, labels


def get_dataset():
    # schema = {
    #             "text": ("var_len", "bytes")
    #             }
    # all_files = tf.io.gfile.glob("gs://legacyai-bucket/c4/en/3.0.1/*.tfrecord*")
    # shuffle(all_files)
    # tf_reader = TFReader(schema=schema, 
    #                                     tfrecord_files=all_files)
    # dataset = tf_reader.read_record(keys=['text'])

    ds = dataset.map(split_fn, num_parallel_calls = tf.data.AUTOTUNE)
    ds = ds.map(filter_empty_string, num_parallel_calls = tf.data.AUTOTUNE)
    ds = ds.filter(filter_sentence_by_count)
    ds = ds.map(prepare_centre_neighbour_sentences, num_parallel_calls = tf.data.AUTOTUNE)
    ds = ds.unbatch()
    ds = ds.batch(batch_size, drop_remainder=True)
    ds = ds.map(shuffle_locally, num_parallel_calls=tf.data.AUTOTUNE)
    #ds = ds.map(create_softmax_mask, num_parallel_calls = tf.data.AUTOTUNE)
    ds = ds.map(text_to_features, num_parallel_calls = tf.data.AUTOTUNE)

    return ds

In [9]:
schema = {
            "text": ("var_len", "bytes")
            }
all_files = tf.io.gfile.glob("gs://legacyai-bucket/c4/en/3.0.1/*.tfrecord*")
shuffle(all_files)
tf_reader = TFReader(schema=schema, 
                                    tfrecord_files=all_files)
dataset = tf_reader.read_record(keys=['text'])

In [63]:
for item in dataset:
  for text in split_fn(item):
    print(text)
  break

tf.Tensor(
[b'Beginners BBQ Class Taking Place in Missoula!\nDo you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now'
 b'Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers'
 b'He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.\nHe will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.\nThe cost to be in the class is $35 per person, and for spectators it is free'
 b'Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared.'], shape=(4,), dtype=string)


Create **Model**

In [85]:
import tensorflow as tf
from tf_transformers.core import LegacyLayer, LegacyModel
from tf_transformers.utils import tf_utils


class Similarity_Model_Pretraining(LegacyLayer):
    def __init__(
        self,
        encoder,
        projection_dimension,
        decoder=None,
        is_training=True,
        use_dropout=False,
        initializer="glorot_uniform",
        siamese=True,
        **kwargs,
    ):
        super(Similarity_Model_Pretraining, self).__init__(
            is_training=is_training, use_dropout=use_dropout, name=encoder.name, **kwargs
        )
        self.is_training = is_training
        if siamese:
            self.encoder = encoder
            self.decoder = encoder
        else:
            if decoder is None:
                raise ValueError("When siamese = False, decoder has to be provided. Provided decoder = None.")
            self.encoder = encoder
            self.decoder = decoder

        self.linear_projection = tf.keras.layers.Dense(
            units=projection_dimension,
            activation=None,
            kernel_initializer=initializer,
            name="linear_projection",
        )

        # As per CLIP paper
        self.logits_scale = tf.Variable(tf.math.log(1 / 0.07), name='logits_scale')


    def get_mean_embeddings(self, token_embeddings, input_mask):
        """ """
        # cls_embeddings = token_embeddings[:, 0, :]  # 0 is CLS (<s>)
        # mask PAD tokens
        token_emb_masked = token_embeddings * tf.cast(tf.expand_dims(input_mask, 2), tf.float32)
        total_non_padded_tokens_per_batch = tf.cast(tf.reduce_sum(input_mask, axis=1), tf.float32)
        # Convert to 2D
        total_non_padded_tokens_per_batch = tf.expand_dims(total_non_padded_tokens_per_batch, 1)
        mean_embeddings = tf.reduce_sum(token_emb_masked, axis=1) / total_non_padded_tokens_per_batch
        return mean_embeddings

    def call(self, inputs):
        """Call"""
        centre_inputs = {k.replace("centre_", ""): v for k, v in inputs.items() if k.startswith("centre_")}
        neighbour_inputs = {k.replace("neighbour_", ""): v for k, v in inputs.items() if k.startswith("neighbour_")}

        centre_outputs = self.encoder(centre_inputs)
        neighbour_outputs = self.decoder(neighbour_inputs)

        if 'cls_output' not in centre_outputs:
            centre_outputs['cls_output'] = tf.keras.layers.Lambda(lambda x: tf.squeeze(x[:, 0:1, :], axis=1))(
                centre_outputs['token_embeddings']
            )
        if 'cls_output' not in neighbour_outputs:
            neighbour_outputs['cls_output'] = tf.keras.layers.Lambda(lambda x: tf.squeeze(x[:, 0:1, :], axis=1))(
                neighbour_outputs['token_embeddings']
            )

        centre_sentence_embedding = self.linear_projection(centre_outputs['cls_output'])
        neighbour_sentence_embedding = self.linear_projection(neighbour_outputs['cls_output'])

        centre_sentence_embedding_mean = self.linear_projection(self.get_mean_embeddings(centre_outputs['token_embeddings'], centre_inputs['input_mask']))
        neighbour_sentence_embedding_mean = self.linear_projection(self.get_mean_embeddings(neighbour_outputs['token_embeddings'], neighbour_inputs['input_mask']))

        centre_sentence_embedding_normalized = tf.keras.layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(
            centre_sentence_embedding
        )
        neighbour_sentence_embedding_normalized = tf.keras.layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(
            neighbour_sentence_embedding
        )

        centre_sentence_embedding_mean_normalized = tf.keras.layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(
            centre_sentence_embedding_mean
        )
        neighbour_sentence_embedding_mean_normalized = tf.keras.layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(
            neighbour_sentence_embedding_mean
        )


        # Clamp logits to a max of tf.math.log(100) = 4.6051702 as per CLIP model
        logits_scale = tf.math.exp(self.logits_scale)
        logits_scale = tf.clip_by_value(
            logits_scale, clip_value_min=tf.math.log(1 / 0.07), clip_value_max=4.6051752
        )

        logits = tf.matmul(
                    centre_sentence_embedding_normalized, neighbour_sentence_embedding_normalized, transpose_b=True
                )
        logits = tf.cast(logits_scale, dtype=tf_utils.get_dtype()) * logits

        logits_mean = tf.matmul(
                    centre_sentence_embedding_mean_normalized, neighbour_sentence_embedding_mean_normalized, transpose_b=True
                )
        logits_mean = tf.cast(logits_scale, dtype=tf_utils.get_dtype()) * logits_mean

        scores = tf.matmul(centre_sentence_embedding_normalized,
                centre_sentence_embedding_normalized, transpose_b=True)
        scores_mask = tf.where(tf.equal(scores, tf.linalg.diag_part(scores)), _large_compatible_negative(scores.dtype), tf.cast(0.0, scores.dtype))
        # Reset only diagonal entries back to 1.0
        scores_mask = tf.linalg.set_diag(scores_mask, tf.zeros(shape=(tf.shape(scores_mask)[0])), name='make_diagonal_one')

        outputs = {}
        outputs['logits'] = logits + scores_mask
        outputs['logits_mean'] = logits_mean + scores_mask
        outputs['centre_sentence_embedding_normalized'] = centre_sentence_embedding_normalized
        outputs['neighbour_sentence_embedding_normalized'] = neighbour_sentence_embedding_normalized
        outputs['centre_sentence_embedding_mean_normalized'] = centre_sentence_embedding_mean_normalized
        outputs['neighbour_sentence_embedding_mean_normalized'] = neighbour_sentence_embedding_mean_normalized

        return outputs
        

    def get_model(self):
        if self.is_training:
            inputs = {}
            # Assume encoder and decoder have same input types
            for k, v in self.encoder.input.items():
                inputs["centre_" + k] = v
            for k, v in self.encoder.input.items():
                inputs["neighbour_" + k] = tf.keras.layers.Input(
                    shape=v.shape[1:], batch_size=v.shape[0], dtype=v.dtype, name=v.name.split(":")[0] + "_2"
                )
        else:
            inputs = {}
            # Assume encoder and decoder have same input types
            for k, v in self.encoder.input.items():
                inputs["first_" + k] = v
            for k, v in self.encoder.input.items():
                inputs["second_" + k] = tf.keras.layers.Input(
                    shape=v.shape[1:], batch_size=v.shape[0], dtype=v.dtype, name=v.name.split(":")[0] + "_3"
                )
        layer_output = self(inputs)
        model = LegacyModel(inputs=inputs, outputs=layer_output, name="similarity_model")
        try:
            model.model_config = self.encoder._config_dict
        except:
            model.model_config = self.encoder.model_config
        return model


Define **Functions**

In [86]:
from tf_transformers.models import AlbertModel
from tf_transformers.optimization import create_optimizer

def loss_fn(batch_labels, model_outputs):
  logits_masked = model_outputs['logits'] 
  logits_mean_masked = model_outputs['logits_mean'] 

  labels = tf.range(tf.shape(logits_masked)[0])
  loss_masked = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits_masked)
  loss_mean_masked = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits_mean_masked)
  loss = (loss_masked + loss_mean_masked)/2.0

  return {'loss_cls': loss_masked, 'loss_mean': loss_mean_masked, 'loss': loss  }


def get_optimizer(learning_rate, num_train_steps, 
                  num_warmup_steps, 
                  decay_function='linear', 
                  weight_decay_rate=0.1, 
                  optimizer_type='adamw', 
                  use_constant_lr=False):
    # As in GPT2 paper, end_learning_rate = 0.1 * learning_rate
    end_learning_rate = 0.1 * learning_rate
    def optimizer_fn():
        optimizer, learning_rate_fn = create_optimizer(
            init_lr=learning_rate,
            num_train_steps=num_train_steps,
            num_warmup_steps=num_warmup_steps,
            decay_function=decay_function,
            weight_decay_rate=weight_decay_rate,
            end_learning_rate=end_learning_rate,
            optimizer_type=optimizer_type,
            use_constant_lr=use_constant_lr,
        )
        return optimizer
    return optimizer_fn

def model_fn():
  encoder = AlbertModel.from_pretrained("albert-base-v2")


  decoder_config = AlbertModel.get_config("albert-base-v2")
  decoder_config['num_hidden_layers']= 6
  decoder = AlbertModel.from_config(decoder_config)
  encoder.save_checkpoint("/tmp/albert/", overwrite=True)
  decoder.load_checkpoint("/tmp/albert")

  model = Similarity_Model_Pretraining(encoder=encoder, projection_dimension=768, decoder=decoder, siamese=False)
  model = model.get_model() 

  return model

In [91]:
learning_rate = 2e-5
epochs = 5
steps_per_epoch = 1000
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = 0.1 * num_train_steps
global_norm = 5.0
train_dataset = get_dataset()
optimizer_fn = get_optimizer(learning_rate, num_train_steps, 
                  num_warmup_steps, decay_function='linear')

model_checkpoint_dir = 'gs://legacyai-bucket/sentence2vec_sample_old_delimter_batch512_noshuffle/'

In [92]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [93]:
WANDB_PROJECT = 'sentence2vec'
config_dict = {}
config_dict['learning_rate'] = learning_rate
config_dict['steps_per_epoch'] = steps_per_epoch
config_dict['epochs'] = epochs
config_dict['num_train_steps'] = steps_per_epoch * epochs
config_dict['num_warmup_steps'] = 0.1 * num_train_steps
config_dict['global_norm'] = global_norm
wandb.init(project=WANDB_PROJECT, config=config_dict, sync_tensorboard=True)


learning_rate,▁▃▅▆█████▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄
loss,█▅▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
loss_cls,█▅▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
loss_mean,█▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
learning_rate,1e-05
loss,1.11144
loss_cls,1.11185
loss_mean,1.11102


In [94]:
# Train
training_loss_names = ['loss_cls', 'loss_mean', 'loss']
history = trainer.run(
    model_fn=model_fn,
    optimizer_fn=optimizer_fn,
    train_dataset=train_dataset,
    train_loss_fn=loss_fn,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    model_checkpoint_dir=model_checkpoint_dir,
    batch_size=batch_size,
    training_loss_names=training_loss_names,
    repeat_dataset=True,
    wandb=wandb,
    clip_norm=global_norm,
)

INFO:absl:Make sure `steps_per_epoch` should be less than or equal to number of batches in dataset.
INFO:absl:Policy: ----> float32
INFO:absl:Strategy: ---> <tensorflow.python.distribute.tpu_strategy.TPUStrategyV2 object at 0x7feed5e29fd0>
INFO:absl:Num TPU Devices: ---> 8
INFO:absl:Successful ✅✅: Model checkpoints matched and loaded from /root/.cache/huggingface/hub/tftransformers__albert-base-v2.main.999c3eeace9b4d2c3f2ad87aad4548b3b73ea3cc/ckpt-1
INFO:absl:Successful ✅: Loaded model from tftransformers/albert-base-v2
INFO:absl:Create model from config
INFO:absl:Successful ✅: Saved model at /tmp/albert/ckpt-1
INFO:absl:Successful ✅✅: Model checkpoints matched and loaded from /tmp/albert/ckpt-1
INFO:absl:Using linear optimization warmup
INFO:absl:Using Adamw optimizer
INFO:absl:No ❌❌ checkpoint found in gs://legacyai-bucket/sentence2vec_sample_old_delimter_batch512_noshuffle/
wandb: WARNING When using several event log directories, please call wandb.tensorboard.patch(root_logdir="..."

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'while/Const_1:0' shape=() dtype=int32>, <tf.Tensor 'while/IteratorGetNext:0' shape=(None, 256) dtype=int32>, <tf.Tensor 'while/IteratorGetNext:1' shape=(None, 256) dtype=int32>, <tf.Tensor 'while/IteratorGetNext:2' shape=(None, 256) dtype=int32>, <tf.Tensor 'while/IteratorGetNext:3' shape=(None, 256) dtype=int32>, <tf.Tensor 'while/IteratorGetNext:4' shape=(None, 256) dtype=int32>, <tf.Tensor 'while/IteratorGetNext:5' shape=(None, 256) dtype=int32>]


Epoch 1/4 --- Step 1000/1000 --- total examples 460800: 100%|██████████| 10/10 [08:01<00:00, 48.13s/batch , learning_rate=1.66e-5, loss=0.877, loss_cls=0.879, loss_mean=0.874]
INFO:absl:Model saved at epoch 1 at gs://legacyai-bucket/sentence2vec_sample_old_delimter_batch512_noshuffle/ckpt-1


Epoch 2/4 --- Step 1000/1000 --- total examples 972800: 100%|██████████| 10/10 [06:58<00:00, 41.81s/batch , learning_rate=1.3e-5, loss=0.823, loss_cls=0.823, loss_mean=0.822]
INFO:absl:Model saved at epoch 2 at gs://legacyai-bucket/sentence2vec_sample_old_delimter_batch512_noshuffle/ckpt-2


Epoch 3/4 --- Step 1000/1000 --- total examples 1484800: 100%|██████████| 10/10 [06:58<00:00, 41.85s/batch , learning_rate=9.38e-6, loss=0.817, loss_cls=0.815, loss_mean=0.818]
INFO:absl:Model saved at epoch 3 at gs://legacyai-bucket/sentence2vec_sample_old_delimter_batch512_noshuffle/ckpt-3


In [83]:
for (batch_inputs, batch_labels) in train_dataset:
  print(batch_inputs, batch_labels)
  break

{'centre_input_ids': <tf.Tensor: shape=(128, 256), dtype=int32, numpy=
array([[   2,  469, 3469, ...,    0,    0,    0],
       [   2,  155,  314, ...,    0,    0,    0],
       [   2,   34,  845, ...,    0,    0,    0],
       ...,
       [   2,  206,   16, ...,    0,    0,    0],
       [   2,   14,  125, ...,    0,    0,    0],
       [   2, 6316, 2925, ...,    0,    0,    0]], dtype=int32)>, 'centre_input_mask': <tf.Tensor: shape=(128, 256), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>, 'centre_input_type_ids': <tf.Tensor: shape=(128, 256), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'neighbour_inpu

In [44]:
import datasets

sts = datasets.load_dataset('glue', 'stsb', split='validation')
# normalize the 0 -> 5 range
sts = sts.map(lambda x: {'label': x['label'] / 5.0})

all_sentences = []
for item in sts:
  all_sentences.append(item['sentence1'])

all_sentences = list(set(all_sentences))
len(all_sentences)

1474

In [45]:
tokenizer_layer_eval = AlbertTokenizerTFText.from_pretrained("albert-base-v2", add_special_tokens=True, dynamic_padding=True, max_length=256)
ds_eval = tf.data.Dataset.from_tensor_slices({'text': all_sentences}).batch(batch_size, drop_remainder=False)

INFO:absl:Loading albert-base-v2 tokenizer to /tmp/tftransformers_tokenizer_cache/albert-base-v2/spiece.model


In [32]:
import tqdm

base_model = AlbertModel.from_pretrained("albert-base-v2")

predicted_embeddings = []
for batch_inputs in tqdm.tqdm(ds_eval):
  model_outputs = base_model(tokenizer_layer_eval(batch_inputs))
  predicted_embeddings.append(model_outputs['cls_output'])

predicted_embeddings = tf.concat(predicted_embeddings, axis=0)
predicted_embeddings = tf.keras.layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(predicted_embeddings)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9d13051410>> (for pre_run_cell):


Exception: ignored

INFO:absl:Successful ✅✅: Model checkpoints matched and loaded from /root/.cache/huggingface/hub/tftransformers__albert-base-v2.main.999c3eeace9b4d2c3f2ad87aad4548b3b73ea3cc/ckpt-1
INFO:absl:Successful ✅: Loaded model from tftransformers/albert-base-v2
100%|██████████| 12/12 [00:25<00:00,  2.16s/it]


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9d13051410>> (for post_run_cell):


Exception: ignored

In [33]:

def score_base_model(text):
  query_vec =   base_model(tokenizer_layer_eval({'text': [text]}))['cls_output']
  query_vec =  tf.keras.layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(query_vec)
  scores = tf.matmul(query_vec, predicted_embeddings, transpose_b=True)
  probs, indexes = tf.nn.top_k(scores, k=10)

  for idx, p_index in enumerate(indexes.numpy()[0]):
    print(all_sentences[p_index], '-->', probs.numpy()[0][idx])

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9d13051410>> (for pre_run_cell):


Exception: ignored

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9d13051410>> (for post_run_cell):


Exception: ignored

In [34]:
score_base_model('A man is dancing.')

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9d13051410>> (for pre_run_cell):


Exception: ignored

A man is dancing. --> 0.9999999
A woman is dancing. --> 0.9996835
A man is running. --> 0.9993486
A man is making a bed. --> 0.9989654
A man is crying. --> 0.9989652
A woman is reading. --> 0.9987549
A man plays the piano. --> 0.9985733
A woman is cutting potato. --> 0.9985633
A woman is taking a picture. --> 0.99834013
A man is speaking. --> 0.9983108
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9d13051410>> (for post_run_cell):


Exception: ignored

In [36]:
score_base_model('Scientists prove there is water on Mars.')

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9d13051410>> (for pre_run_cell):


Exception: ignored

Boat sails in rough water. --> 0.97101307
My answer would be depending on which GRE are you referring to? --> 0.9644584
Stars form in star-formation regions, which itself develop from molecular clouds. --> 0.95424724
No, you don't need to have taken classes or earned a degree in your area. --> 0.9532987
The Hare Psychopathy Checklist is often used to assess psychopathy in clinical settings. --> 0.95270234
The assertion is false, at least as you've restated it here. --> 0.95083284
Pick them as soon as they are as ripe as you want them. --> 0.9498982
The answer will depend significantly on what country you're traveling to. --> 0.9497186
The player shoots the winning points. --> 0.94791687
Living room or den with large screen television. --> 0.94762164
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9d13051410>> (for post_run_cell):


Exception: ignored

In [22]:
import tqdm

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9d13051410>> (for pre_run_cell):


Exception: ignored

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9d13051410>> (for post_run_cell):


Exception: ignored

In [46]:
finetuned_model = model_fn()
finetuned_model.load_checkpoint("gs://legacyai-bucket/sentence2vec_sample/")
finetuned_model = finetuned_model.layers[-1].encoder

INFO:absl:Successful ✅✅: Model checkpoints matched and loaded from /root/.cache/huggingface/hub/tftransformers__albert-base-v2.main.999c3eeace9b4d2c3f2ad87aad4548b3b73ea3cc/ckpt-1
INFO:absl:Successful ✅: Loaded model from tftransformers/albert-base-v2
INFO:absl:Create model from config
INFO:absl:Successful ✅: Saved model at /tmp/albert/ckpt-1
INFO:absl:Successful ✅✅: Model checkpoints matched and loaded from /tmp/albert/ckpt-1
INFO:absl:Successful ✅✅: Model checkpoints matched and loaded from gs://legacyai-bucket/sentence2vec_sample/ckpt-1


In [47]:
import tqdm

In [48]:
predicted_embeddings_finetuned = []
for batch_inputs in tqdm.tqdm(ds_eval):
  model_outputs = finetuned_model(tokenizer_layer_eval(batch_inputs))
  predicted_embeddings_finetuned.append(model_outputs['cls_output'])

predicted_embeddings_finetuned = tf.concat(predicted_embeddings_finetuned, axis=0)
predicted_embeddings_finetuned = tf.keras.layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(predicted_embeddings_finetuned)

100%|██████████| 12/12 [00:26<00:00,  2.19s/it]


In [49]:

def score_finetuned_model(text):
  query_vec =   finetuned_model(tokenizer_layer_eval({'text': [text]}))['cls_output']
  query_vec =  tf.keras.layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(query_vec)
  scores = tf.matmul(query_vec, predicted_embeddings_finetuned, transpose_b=True)
  probs, indexes = tf.nn.top_k(scores, k=10)

  for idx, p_index in enumerate(indexes.numpy()[0]):
    print(all_sentences[p_index], '-->', probs.numpy()[0][idx])

In [50]:
score_finetuned_model("A man is dancing")

A man is dancing. --> 0.9917348
A man with a hard hat is dancing. --> 0.9883332
A woman is dancing. --> 0.9742526
Three men are dancing. --> 0.95932573
A woman is playing the flute. --> 0.9564247
A woman is playing the guitar. --> 0.94910264
A man is crying. --> 0.9489113
A man and woman blow bubbles. --> 0.9484095
A man plays the piano. --> 0.9411945
A woman runs on the beach. --> 0.94074315


In [51]:
score_finetuned_model('Scientists prove there is water on Mars')

Scientists prove there is water on Mars --> 0.9999999
Asteroids are what are called minor planets - small inner solar system rocky bodies. --> 0.91068494
Two astronomers surveying the region around Jupiter have detected 20 new moons, bringing the giant planet's total to 60. --> 0.88290787
A black hole (briefly), is nothing more than a dead star whose mass was more than 3 solar masses. --> 0.87302935
The farther away, the faster the galaxies move away from us. --> 0.8227091
The sky has very little to no clouds. --> 0.8211899
A turtle walks over the ground. --> 0.80502135
The identical rovers will act as robotic geologists, searching for evidence of past water. --> 0.7918004
A group of people are camping out on some rocks. --> 0.79133594
Chinese lunar rover lands on moon --> 0.7847755


In [52]:
score_finetuned_model('The cookies was delicious.')

The lady made some slices on the shrimp. --> 0.9673203
I just hold the cheese closer to the edge away from the grater. --> 0.8668589
The lady cracked an egg for the mixer. --> 0.8489798
These cooks in the white are busy in the kitchen making dinner for their customers. --> 0.8274094
A chef is slicing carrot. --> 0.8228042
A chef is preparing some food. --> 0.81544566
I wrap steaks individually in freezer paper, then put them into ziploc freezer bags. --> 0.8087766
The lady put the skewered shimp in the hot water. --> 0.79851776
The recipe I work from has you put the meat in the freezer, then pan sear it. --> 0.79590166
The cook is kneading the flour. --> 0.79156244


In [53]:
score_finetuned_model('I had a wonderful vacation with my family.')

I visited Peru in August, starting from Lima and touring for three weeks. --> 0.90224445
When I needed a travel tripod, I also opted for a Trekpod. --> 0.87706655
I visit AZ every summer and include some hikes in the desert when I'm there. --> 0.72904265
Our daughter has always loved her bath, but coming out... not so much. --> 0.7203518
Reading your post, made me think of my younger years !! --> 0.7175178
Two girls walking in the ocean. --> 0.70754194
My experience is that a normal taxi is always cheaper. --> 0.7055931
Oh friend, I have been down this miserable road just like you. --> 0.70546025
I would say the best thing for me is to have a proper and consistent sleep schedule. --> 0.7023461
As I remember, in Geneva at least, there are open air markets on a weekly basis. --> 0.6924932


In [54]:
score_finetuned_model("The flight ticket was cheap")

My experience is that a normal taxi is always cheaper. --> 0.8690753
When I needed a travel tripod, I also opted for a Trekpod. --> 0.8610847
Would going on a cargo ship count as a tour? --> 0.83855987
An AeroMexico jet taxing along a runway. --> 0.8321451
A Das Air Cargo plane sits on the runway. --> 0.82384926
As I remember, in Geneva at least, there are open air markets on a weekly basis. --> 0.82079893
There is one airline, Spirit Airlines, which is trying to copy Ryanair with super cheap tickets and aggressively bad service. --> 0.8071083
A white plane on the runway. --> 0.80360734
A technical stop is for the benefit of the PLANE. --> 0.8018749
Two airplanes in the sky. --> 0.78709626


In [55]:
score_finetuned_model("The taxi was cheap")

My experience is that a normal taxi is always cheaper. --> 0.8610263
When I needed a travel tripod, I also opted for a Trekpod. --> 0.82155836
An AeroMexico jet taxing along a runway. --> 0.81527424
Would going on a cargo ship count as a tour? --> 0.80877316
A Das Air Cargo plane sits on the runway. --> 0.8077455
A white plane on the runway. --> 0.7918957
Two airplanes in the sky. --> 0.78370345
As I remember, in Geneva at least, there are open air markets on a weekly basis. --> 0.7748461
The people are leaving the airplane. --> 0.76586545
There is one airline, Spirit Airlines, which is trying to copy Ryanair with super cheap tickets and aggressively bad service. --> 0.7657012


In [37]:
score_base_model("The taxi was cheap")

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9d13051410>> (for pre_run_cell):


Exception: ignored

man on steps --> 0.98259246
Two people in snowsuits laying in the snow making snow angels --> 0.9818696
A group wading across a witch --> 0.97996974
A man jumping rope --> 0.9797159
A white dog is running through the water at a beach --> 0.9790625
a wet black dog with a red collar is in the river --> 0.9782449
a young girl smiling with her head upside down --> 0.9778154
the man is catching a ball --> 0.9773327
Three men are dancing. --> 0.97712135
Friends walk into a building --> 0.97696257
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9d13051410>> (for post_run_cell):


Exception: ignored

In [28]:
for item in dataset:
  print(item)
  break

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9d13051410>> (for pre_run_cell):


Exception: ignored

{'text': <tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'Beginners BBQ Class Taking Place in Missoula!\nDo you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers. He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.\nHe will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.\nThe cost to be in the class is $35 per person, and for spectators it is free. Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared.'],
      dtype=object)>}
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9d13051410>> (for post_run_cell):


Exception: ignored

In [ ]:
 delim_regex_pattern = r'\.\s?\n?'
 text_splitted = tf_text.regex_split(
      item['text'][0], delim_regex_pattern, keep_delim_regex_pattern=' ', name='split'
  )
for item in text_splitted:
  print(item)

tf.Tensor(
[b'Beginners BBQ Class Taking Place in Missoula!\nDo you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now'
 b'Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers'
 b'He will be teaching a beginner level class for everyone who wants to get better with their culinary skills'
 b'He will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information'
 b'The cost to be in the class is $35 per person, and for spectators it is free'
 b'Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared'], shape=(6,), dtype=string)
